In [1]:
import pandas as pd
import requests, re, time, os
from bs4 import BeautifulSoup 

#### 나무위키의 최근 변경내역 페이지 정보 수집

In [2]:
# 주소.
site = 'http://namu.wiki/RecentChanges'

# 요청.
res = requests.get(site)
html = res.text
soup = BeautifulSoup(html, 'lxml')
# soup

In [3]:
# 데이터 수집.
table_tag   = soup.select_one('#app > div > div:nth-child(2) > article > div > table')
tbody_tag   = table_tag.select_one('tbody')
tr_tag_List = tbody_tag.select('tr')


# Link 주소를 담을 리스트 생성.
Link_List = []

# tr tag의 수 만큼 반복.
for tr_tag in tr_tag_List :
    # tr_tag 내에서 td_tag를 가져옴.
    td_List = tr_tag.select('td')
    
    # 추출한 td_tag 중 첫번째 것만 이용.
    td_tag = td_List[0]
    # print(td_tag)
    # print('-'*130)
    
    # td_tag 내의 a_tag를 가져옴.
    a_tag_List = td_tag.select('a')
    # print(a_tag_List)
    # print('-'*130)
    
    # 가져온 a_tag 가 존재할 경우만 작업.
    if len(a_tag_List) > 0 :
        # 첫 번째 a_tag의 href 속성값을 가져옴.
        href = a_tag_List[0].get('href')
        # print(href)
        
        # 리스트에 담기.
        Link_List.append(href)

Link_List    

['/w/%EC%B9%B4%EB%A9%9C%EB%A1%AF(%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8)',
 '/w/%EB%A7%8C%EC%95%85%EC%9D%98%20%EA%B7%BC%EC%9B%90/%EC%B0%BD%EC%9E%91%EB%AC%BC',
 '/w/%EB%8D%B0%EB%AF%B8%EC%95%88(%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8)',
 '/w/Merry%20Christmas',
 '/w/%EC%9D%B4%EB%8F%84%ED%9D%AC',
 '/w/%EC%9C%A0%EB%A6%AC%20%EA%B0%80%EA%B0%80%EB%A6%B0',
 '/w/Apex%20%EB%A0%88%EC%A0%84%EB%93%9C/%EC%95%84%EC%9D%B4%ED%85%9C',
 '/w/%EA%B4%80%EC%9A%B0(%EC%82%BC%EA%B5%AD%EC%A7%80%ED%86%A1)',
 '/w/%ED%92%80%EB%9D%BC%EC%96%B4',
 '/w/BloxVerse:%20Unlimited',
 '/w/%EC%97%B4%ED%95%98%EC%9D%BC%EA%B8%B0',
 '/w/%EC%98%A4%EC%8A%A4%ED%8A%B8%EB%A6%AC%EC%95%84-%ED%97%9D%EA%B0%80%EB%A6%AC%20%EC%A0%9C%EA%B5%AD',
 '/w/%ED%82%A4%EC%82%AC%EB%9D%BC%EA%B8%B0%20%EC%B9%98%ED%95%98%EC%95%BC',
 '/w/%EC%82%AC%EC%9D%B4%EB%B2%84%20%EB%8B%A4%ED%81%AC%20%EC%97%94%EB%93%9C%20%EB%93%9C%EB%9E%98%EA%B3%A4',
 '/w/%EC%9D%B4%EB%8F%84%ED%9D%AC',
 '/w/%EC%98%81%EA%B5%AD%EC%8B%9D%20%EC%98%81%EC%96%B4',
 

In [4]:
# 주소의 수 만큼 반복.
for sub_url in Link_List :
    time.sleep(1)
    
    # 데이터를 담을 딕셔너리 생성.
    data_dict = {
            'title' : [],
            'content' : []  
            }
    
    # 페이지 내의 데이터 수집.
    # 요청 주소2.
    site2 = f'http://namu.wiki{sub_url}'
    # print(site2)
    print(f'{site2} 수집중')

    # 요청.
    res2 = requests.get(site2)
    soup2 = BeautifulSoup(res2.text, 'lxml')

    # 글 전체를 가져옴.
    article_tag = soup2.select_one('#app > div > div:nth-child(2) > article')

    # 글을 가지고 있는 tag들을 가져옴.
    content_paragraphs = article_tag.select('div.wiki-paragraph')

    # 글의 수 만큼 반복. 
    # 글 내용을 하나의 문자열로 합침.
    Content_text = ''
    for content_tag in content_paragraphs :
        Content_text = Content_text + content_tag.text

    # 제거하고자 하는 문자열들 제거.
    Content_text = Content_text.replace('번 문단', '')
    Content_text = Content_text.replace(' 문서의 r 판', '')
    Content_text = Content_text.replace('(이전 역사)', '')
    Content_text = Content_text.replace('\xa0', '')
    Content_text = Content_text.replace(',', '')
    Content_text = Content_text.strip()
    
     # 제목을 가져옴.
    h1_tag = article_tag.select_one('h1')
    title_text = h1_tag.text.strip()
    # print(title_text)
    # print(Content_text)
    
    data_dict['title'].append(title_text)
    data_dict['content'].append(Content_text)
    
    # 데이터 프레임 생성.
    df = pd.DataFrame(data_dict)
    # display(df1)
    
    if os.path.exists('namuwiki.csv') == False :
        # 파일이 없을 경우.
        df.to_csv('namuwiki.csv', encoding='utf-8-sig', index=False)
    else :
        df.to_csv('namuwiki.csv', encoding='utf-8-sig', index=False, header=False, mode='a')   
print('저장완료')

http://namu.wiki/w/%EC%B9%B4%EB%A9%9C%EB%A1%AF(%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8) 수집중
http://namu.wiki/w/%EB%A7%8C%EC%95%85%EC%9D%98%20%EA%B7%BC%EC%9B%90/%EC%B0%BD%EC%9E%91%EB%AC%BC 수집중
http://namu.wiki/w/%EB%8D%B0%EB%AF%B8%EC%95%88(%EC%8A%A4%ED%83%80%ED%81%AC%EB%9E%98%ED%94%84%ED%8A%B8) 수집중
http://namu.wiki/w/Merry%20Christmas 수집중
http://namu.wiki/w/%EC%9D%B4%EB%8F%84%ED%9D%AC 수집중
http://namu.wiki/w/%EC%9C%A0%EB%A6%AC%20%EA%B0%80%EA%B0%80%EB%A6%B0 수집중
http://namu.wiki/w/Apex%20%EB%A0%88%EC%A0%84%EB%93%9C/%EC%95%84%EC%9D%B4%ED%85%9C 수집중
http://namu.wiki/w/%EA%B4%80%EC%9A%B0(%EC%82%BC%EA%B5%AD%EC%A7%80%ED%86%A1) 수집중
http://namu.wiki/w/%ED%92%80%EB%9D%BC%EC%96%B4 수집중
http://namu.wiki/w/BloxVerse:%20Unlimited 수집중
http://namu.wiki/w/%EC%97%B4%ED%95%98%EC%9D%BC%EA%B8%B0 수집중
http://namu.wiki/w/%EC%98%A4%EC%8A%A4%ED%8A%B8%EB%A6%AC%EC%95%84-%ED%97%9D%EA%B0%80%EB%A6%AC%20%EC%A0%9C%EA%B5%AD 수집중
http://namu.wiki/w/%ED%82%A4%EC%82%AC%EB%9D%BC%EA%B8%B0%20%EC%B9%98%ED%95%98%EC%9

In [6]:
df1 = pd.read_csv('namuwiki.csv')
df1.head()

,title,content
0,카멜롯(스타크래프트),시즌3 대회 본선 맵 카멜롯 안드로메다 아웃사이더 SE 서킷브레이커...
1,만악의 근원/창작물,상위 문서: 만악의 근원1. 개요2. 목록2.1. 보편적2.2. 장르별2.3. 작품...
2,데미안(스타크래프트),시즌2 대회 본선 맵 서킷브레이커 태풍의 눈 벤젠 데미안 데미안(Demi...
3,Merry Christmas,자세한 내용은 머라이어 캐리/음반 목록 문서를의 번째 문단을의 부분을 참고하십시오...
4,이도희,이도희의 수상 및 감독 경력 [ 펼치기 · 접기 ] 대한민국 여자 배구 국가대표...
